5.1.1 自定义块

In [3]:
import torch
from torch import nn
from torch.nn import functional as F


In [4]:
# 多层感知机

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

x = torch.rand(2, 20)
net(x)

tensor([[ 3.6363e-02,  2.5111e-01, -1.4637e-01,  8.3014e-05, -1.2809e-01,
          4.4949e-02, -9.3296e-02,  2.9547e-03,  1.2905e-01, -1.6684e-01],
        [-6.1763e-02,  3.3237e-01, -2.1787e-01,  5.3973e-03, -1.8795e-01,
          8.8189e-02, -1.3969e-01, -9.3340e-02,  1.9851e-01, -1.9477e-01]],
       grad_fn=<AddmmBackward0>)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

net = MLP()
net(x)

tensor([[ 0.2366, -0.0809, -0.0487,  0.1132, -0.0119, -0.0639,  0.0087,  0.0150,
         -0.1293,  0.0700],
        [ 0.2026, -0.0300, -0.0575,  0.0799,  0.0614, -0.1144, -0.0461, -0.0769,
         -0.1513,  0.1596]], grad_fn=<AddmmBackward0>)

5.1.2 顺序块

In [17]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in  enumerate(args):
            self._modules[str(idx)] = module
        
    def forward(self, x):
        for blcok in self._modules.values():
            x = blcok(x)
        return x

In [56]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

net(x)

tensor([[-0.0409, -0.0353, -0.1685,  0.0885,  0.0943,  0.0599, -0.0645, -0.1565,
          0.1460, -0.1376],
        [ 0.0232,  0.0307, -0.1977,  0.1287,  0.0541,  0.0176, -0.1485, -0.1485,
          0.1949, -0.1337]], grad_fn=<AddmmBackward0>)

5.1.3 在前向传播函数中执行代码

In [59]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x =self.linear(x)

        x = F.relu(torch.mm(x, self.rand_weight) + 1)

        x = self.linear(x)

        while x.abs().sum() > 1:
            x /=2
        return x.sum()

In [60]:
net = FixedHiddenMLP()
net(x)

tensor(-0.1595, grad_fn=<SumBackward0>)

In [62]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential( nn.Linear(20 ,64), nn.ReLU(), 
                                                            nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, x):
        return self.linear(self.net(x))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16 , 20), FixedHiddenMLP() )

chimera(x)

tensor(0.2179, grad_fn=<SumBackward0>)